# Cpt2. Directed Graphical Models

## Graph terminology

![Image](figures/2.png)
![Image](figures/3.png)

Node / Vertices: 교점/ 꼭지점(vertex)

    V = {V1 ,V2 ,...Vn }

Edges: 모서리    

    E = {E1 , E2 ,... En }
    Directed Edge: 방향성이 있는 것
    Undirected Egde: 방향성이 없는 것
    - 인접행렬로 나타낼 수 있다.(1/0, -1/0/1)

Directed Graphs: Directed Edges로만 이루어진 그래프
    
    Children: V1에서 나간 edges이 도착하는 노드들의 집합
    Parents: V1에 도착하는 edges이 출발한 노드들의 집합
    Degree of nodes: 그 노드에 참여하고 있는 edges의 갯수
    
Clique: 모든 노드들끼리 엣지로 연결되어 있는 노드들의 집합
    
    Maximal clique: 새로운 노드를 추가할 때 Clique의 성격을 잃어버리는 수
  
Cycle / Loop: 순환/ 고리

    노드가 다른 노드들을 거쳐 그 노드로 돌아오는 상황
    
DAG (Directed Acyclic Graph)

    모두 directed edges이고 cycle이 없는 그래프
    
PDAG (Partially Directed Acyclic Graph)

    Directed와 Undirected를 모두 가지면서 cycle이 없는 그래프   

## Python digression

pip install "packagename"

Libpgm / Scipy 를 설치

## Independence and independent parameters

우리의 주요 문제는 Joint Distribution을 정의하는 것이다. 

![Image](figures/4.png)

1) 모든 순열값들을 보여주기

    Ex) 변수가 Experience / Grade / Interview / Offer인데
    Experience(Good/Bad:2)
    Grade(High/Low:2)
    Interview(0/1/2:3)
    Offer(Yes/No:2)
 =>2X2X3X2=24개의 행을 얻게 되지만
 
 모든 확률값들의 합은 4이므로 (4가지 경우의 수가 6가지로 나눠지는 방식) 1개는 필요없으므로 24-1=23행이 필요하게 된다.
 
    문제점 
-	보관하고 조작하기에 너무 크다
-	각 조합확률값을 제대로 끌어내려면 많은 데이터가 필요하다
-	큰 조합분포의 각 확률 값은 인간이 해석하기에 너무 작아져 버릴 수 있다. 

2) Independent Parameters를 사용하는 방법
    ![Image](figures/5.PNG)
    
    Ex1) 성적과 합격 변수만 있을 경우
    모든 순열값들을 보여주면 4-1=3가지 경우를 보여주어야 한다.
    
    조건부확률을 이용해서 다음과같이 나타낼 수 있다.
    그렇다면 변수 값은 P(S), P(A|S0), P(A|S1)세 가지 값이 필요하다.
    
    
![Image](figures/6.png)
    
    Ex2) 위의 예시를 보면
    P(Experience), P(Grade)각각 하나
    P(Interview=z|E=x,G=y) - 8개 (2x2x2)
    P(Offer=w|interview=z) - 3개 (3x1)
    =>1+1+8+3=13개의 값만 필요하다. (23개보다 작다!)

------------------
## The Bayes network

베이즈 네트워크란 DAG로 표현될 수 있는 구조이고 
두가지 관점으로 볼 수 있다.
1) 체인룰을 이용하여 베이즈 네트워크를 위하여 간단하고 모듈화된 Joint Distribution을 표현할 수 있다.
모듈화된 표현이란 각 노드간의 parents/childres와 같은 의존도를 나타내는 방식이다.

    Ex)
    확률 분포
    P(Experience)
    P(Grade)

    조건부 확률 분포 (CPD:Conditional Probability DIstribution)
    P(Interview=z|E=x,G=y)
    P(Offer=w|interview=z)

2) 관찰한 노드들 간의 조건부 독립 가정을 가능하게 한다. 
조건부 독립 관점이란 변수들 간의 조건부 독립성을 확인할 수 있다는 의미이다.

    Ex) 위 예시에서 인터뷰는 경험과 점수에 종속되어있고 가정한다. 또한 제안은 인터뷰에만 종속되어있다고 가정한다. 

### Chain Rule

기본적인 체인 룰의 식은 다음과 같다.

![Image](figures/7.PNG)

하지만 우리가 조건부 독립 가정을 한다면 이렇게 바뀔 수 있다.

![Image](figures/8.PNG)


일반화된 식으로는

![Image](figures/9.PNG)

(Par은 그 노드의 Parent노드들을 의미한다)

------------------------------------------------

## Reasoning patterns

3가지 추론방법이 있다

기본적 코드 세팅- 패키지를 불러온다

In [1]:
from libpgm.graphskeleton import GraphSkeleton
from libpgm.nodedata import NodeData
from libpgm.discretebayesiannetwork import DiscreteBayesianNetwork
from libpgm.tablecpdfactorization import TableCPDFactorization

def getTableCPD():
    nd = NodeData()
    skel = GraphSkeleton()
    jsonpath = "job_interview.txt"
    nd.load(jsonpath)
    skel.load(jsonpath)
    # load bayesian network
    bn = DiscreteBayesianNetwork(skel, nd)
    tablecpd = TableCPDFactorization(bn)
    return tablecpd

![Image](figures/10.PNG)

### Causal reasoning

우리가 Parent(원인)의 사건이 관찰될 때 Child(결과)의 확률에 대한 신뢰가 강해진다. 

Ex) P(Offer = 1 )

In [2]:
tcpd = getTableCPD()
tcpd.specificquery(dict(Offer='1'), dict())

0.432816

P (Offer = 1 | Grades = 0 )

In [3]:
tcpd=getTableCPD()
tcpd.specificquery(dict(Offer='1'),dict(Grades='0'))

0.35148

P(Offer =1|Grades = 0,Experience = 0),

In [4]:
tcpd=getTableCPD()
tcpd.specificquery(dict(Offer='1'),dict(Grades='0',Experience='0'))

0.2078

### Evidential reasoning

우리가 Child(결과)의 값을 관찰할 때 Parent(원인)에 대한 믿음이 바뀌는 정도를 추론한다.

P(Experience =1)

In [5]:
tcpd=getTableCPD()
tcpd.specificquery(dict(Experience='1'),dict())

0.4000000000000001

P(Experience=1|Interview=2)

In [6]:
tcpd=getTableCPD()
tcpd.specificquery(dict(Experience='1'),dict(Interview='2'))

0.8641975308641975

### Inter-causal reasoning

![image](figures/11.PNG)

한 가지 Parent(결과)에 영향을 미치는 다양한 Children(원인들)의 관계를 추론한다. 

P(Experience=1)

In [7]:
tcpd=getTableCPD()
tcpd.specificquery(dict(Experience='1'),dict())

0.4000000000000001

P(Experience =1| Interview = 2)

In [8]:
tcpd=getTableCPD()
tcpd.specificquery(dict(Experience='1'),dict(Interview='2'))


0.8641975308641975

P(Experience =1| Interview = 2,Grades = 0)